# Import Module

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update
!apt-get install tensorflow-model-server

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  30656      0 --:--:-- --:--:-- --:--:-- 30656
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1

In [ ]:
import os
import json
import tempfile
import requests
import subprocess

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
MODEL_DIR = tempfile.gettempdir()
def gen_export_path(ver):    
    export_path = os.path.join(MODEL_DIR, str(ver))
    return export_path 

# Build Model

In [ ]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train, X_test = np.expand_dims(X_train, -1), np.expand_dims(X_test, -1) # expand (height x weight) to (height x weight x color), color = 1 for grayscale
print(X_train.shape, X_test.shape)

K = len(set(y_train))
print(K)

4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28, 1) (10000, 28, 28, 1)
10


In [ ]:
# Label mapping
labels = '''T-shirt/top
Trouser
Pullover
Dress
Coat
Sandal
Shirt
Sneaker
Bag
Ankle boot'''.split("\n")

## Model - Large

In [ ]:
i = Input(shape=X_train[0].shape)
x = Conv2D(32, (3, 3), strides=2, activation='relu')(i)
x = Conv2D(64, (3, 3), strides=2, activation='relu')(x)
x = Conv2D(128, (3, 3), strides=2, activation='relu')(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation='softmax')(x)

model = Model(i, x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 13, 13, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
r = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15)

Epoch 1/15
1875/1875 [==============================] - 38s 3ms/step - loss: 0.5310 - accuracy: 0.8031 - val_loss: 0.4156 - val_accuracy: 0.8473
Epoch 2/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3652 - accuracy: 0.8631 - val_loss: 0.3732 - val_accuracy: 0.8578
Epoch 3/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3158 - accuracy: 0.8801 - val_loss: 0.3208 - val_accuracy: 0.8835
Epoch 4/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2813 - accuracy: 0.8944 - val_loss: 0.3157 - val_accuracy: 0.8837
Epoch 5/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2567 - accuracy: 0.9023 - val_loss: 0.3124 - val_accuracy: 0.8884
Epoch 6/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2368 - accuracy: 0.9107 - val_loss: 0.3075 - val_accuracy: 0.8892
Epoch 7/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2173 - accuracy: 0.9183 - val_loss: 0.3003 - val_accuracy

In [ ]:
export_path = gen_export_path(1)
tf.saved_model.save(model, export_path)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: /tmp/1/assets


## Model - Tiny

In [ ]:
i = Input(shape=X_train[0].shape)
x = Conv2D(32, (3, 3), strides=2, activation='relu')(i)
x = Flatten()(x)
x = Dense(K, activation='softmax')(x)

model2 = Model(i, x)
model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 32)        320       
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
r2 = model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15)

Epoch 1/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4646 - accuracy: 0.8370 - val_loss: 0.3757 - val_accuracy: 0.8670
Epoch 2/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3285 - accuracy: 0.8848 - val_loss: 0.3405 - val_accuracy: 0.8787
Epoch 3/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2950 - accuracy: 0.8947 - val_loss: 0.3169 - val_accuracy: 0.8849
Epoch 4/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2730 - accuracy: 0.9020 - val_loss: 0.3124 - val_accuracy: 0.8866
Epoch 5/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2569 - accuracy: 0.9082 - val_loss: 0.3066 - val_accuracy: 0.8886
Epoch 6/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2436 - accuracy: 0.9126 - val_loss: 0.2995 - val_accuracy: 0.8937
Epoch 7/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2307 - accuracy: 0.9165 - val_loss: 0.3091 - val_accuracy:

In [ ]:
export_path = gen_export_path(2)
tf.saved_model.save(model2, export_path)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: /tmp/2/assets


In [ ]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_input_2:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0629 17:27:51.879715 139632864188288 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/re

# Prepare TF Server

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [ ]:
!tail server.log

2021-06-29 17:15:41.249794: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /tmp/1
2021-06-29 17:15:41.257079: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 70159 microseconds.
2021-06-29 17:15:41.258038: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2021-06-29 17:15:41.258159: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: fashion_model version: 1}
2021-06-29 17:15:41.258721: I tensorflow_serving/model_servers/server_core.cc:486] Finished adding/updating models
2021-06-29 17:15:41.258780: I tensorflow_serving/model_servers/server.cc:367] Profiler service is enabled
2021-06-29 17:15:41.259166: I tensorflow_serving/model_servers/server.cc:393] Running gRPC ModelServer at 0.0.0.0:8500 ..

# Test TF Server

In [ ]:
data = json.dumps({"signature_name": "serving_default", "instances": X_test[0:3].tolist()})
headers = {"content-type": "application/json"}

In [ ]:
# Model - Large
r = requests.post('http://localhost:8501/v1/models/fashion_model/versions/1:predict', data=data, headers=headers)     # specifically ver1
p = r.json()['predictions']
p = np.array(p).argmax(axis=1)
p = [labels[p_] for p_ in p]
a = [labels[i] for i in y_test[:3]]
print(p, a)

['Ankle boot', 'Pullover', 'Trouser'] ['Ankle boot', 'Pullover', 'Trouser']


In [ ]:
# Model - Tiny
# r = requests.post('http://localhost:8501/v1/models/fashion_model:predict', data=data, headers=headers)  # latest version
r = requests.post('http://localhost:8501/v1/models/fashion_model/versions/2:predict', data=data, headers=headers)     # specifically ver2
p = r.json()['predictions']
p = np.array(p).argmax(axis=1)
p = [labels[p_] for p_ in p]
a = [labels[i] for i in y_test[:3]]
print(p, a)

['Ankle boot', 'Pullover', 'Trouser'] ['Ankle boot', 'Pullover', 'Trouser']


In [ ]:
# non-existent model will return Response 404
requests.post('http://localhost:8501/v1/models/fashion_model/versions/3:predict', data=data, headers=headers)     # specifically ver2

<Response [404]>